This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/shadow_deploy).

## Shadow Deployment Tutorial

Wallaroo provides a method of testing the same data against two different models or sets of models at the same time through **shadow deployments** otherwise known as **parallel deployments**.  This allows data to be submitted to a pipeline with inferences running on two different sets of models.  Typically this is performed on a model that is known to provide accurate results - the **champion** - and a model that is being tested to see if it provides more accurate or faster responses depending on the criteria known as the **challengers**.  Multiple challengers can be tested against a single champion.

As described in the Wallaroo blog post [The What, Why, and How of Model A/B Testing](https://www.wallaroo.ai/blog/the-what-why-and-how-of-a/b-testing):

> In data science, A/B tests can also be used to choose between two models in production, by measuring which model performs better in the real world. In this formulation, the control is often an existing model that is currently in production, sometimes called the champion. The treatment is a new model being considered to replace the old one. This new model is sometimes called the challenger....

> Keep in mind that in machine learning, the terms experiments and trials also often refer to the process of finding a training configuration that works best for the problem at hand (this is sometimes called hyperparameter optimization).

For Arrow enabled Wallaroo instances the inference result is returned as a pandas DataFrame.  The shadow results from the models is stored in the `out_{model name}.variable` column.  For example, the model above named `ccfraudrf` would have its results in the column `out_ccfraudrf.variable`.

For non-Arrow enabled Wallaroo instances, the inference result is as a Wallaroo InferenceResult object.  The inference results from the `champion` model will be available in the returned InferenceResult Object's `data` element, while inference results from each of the `challenger` models will be in the returned InferenceResult Object's `shadow_data` element.

The following tutorial will demonstrate how:

* Upload champion and challenger models into a Wallaroo instance.
* Create a shadow deployment in a Wallaroo pipeline.
* Perform an inference through a pipeline with a shadow deployment.
* View the `data` and `shadow_data` results from the InferenceResult Object.
* View the pipeline logs and pipeline shadow logs.

This tutorial provides the following:

* `dev_smoke_test.json`:  Sample test data used for the inference testing.
* `models/keras_ccfraud.onnx`:  The champion model.
* `models/modelA.onnx`: A challenger model.
* `models/xgboost_ccfraud.onnx`: A challenger model.

All models are similar to the ones used for the Wallaroo-101 example included in the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials).

## Steps

### Import libraries

The first step is to import the libraries required.


In [19]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to Wallaroo

Connect to your Wallaroo instance and save the connection as the variable `wl`.

In [20]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [21]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
# os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

True


### Set Variables

The following variables are used to create or use existing workspaces, pipelines, and upload the models.  Adjust them based on your Wallaroo instance and organization requirements.

In [22]:
workspace_name = 'ccfraudcomparisondemo'
pipeline_name = 'cc-shadow'
champion_model_name = 'ccfraudlstm'
champion_model_file = 'models/keras_ccfraud.onnx'
shadow_model_01_name = 'ccfraudxgb'
shadow_model_01_file = 'models/xgboost_ccfraud.onnx'
shadow_model_02_name = 'ccfraudrf'
shadow_model_02_file = 'models/modelA.onnx'

### Workspace and Pipeline

The following creates or connects to an existing workspace based on the variable `workspace_name`, and creates or connects to a pipeline based on the variable `pipeline_name`.

In [23]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [24]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
pipeline


name,cc-shadow
created,2023-03-01 19:29:44.071174+00:00
last_updated,2023-03-01 19:29:44.071174+00:00
deployed,(none)
tags,
versions,eaa129cb-d505-46fd-95e2-91b09bc2c02c
steps,


### Load the Models

The models will be uploaded into the current workspace based on the variable names set earlier and listed as the `champion`, `model2` and `model3`.

In [25]:
champion = wl.upload_model(champion_model_name, champion_model_file).configure()
model2 = wl.upload_model(shadow_model_01_name, shadow_model_01_file).configure()
model3 = wl.upload_model(shadow_model_02_name, shadow_model_02_file).configure()

### Create Shadow Deployment

A shadow deployment is created using the `add_shadow_deploy(champion, challengers[])` method where:

* `champion`: The model that will be primarily used for inferences run through the pipeline.  Inference results will be returned through the Inference Object's `data` element.
* `challengers[]`: An array of models that will be used for inferences iteratively.  Inference results will be returned through the Inference Object's `shadow_data` element.

In [26]:
pipeline.add_shadow_deploy(champion, [model2, model3])

name,cc-shadow
created,2023-03-01 19:29:44.071174+00:00
last_updated,2023-03-01 19:29:44.071174+00:00
deployed,(none)
tags,
versions,eaa129cb-d505-46fd-95e2-91b09bc2c02c
steps,


In [27]:
pipeline.deploy()

name,cc-shadow
created,2023-03-01 19:29:44.071174+00:00
last_updated,2023-03-01 19:29:49.728761+00:00
deployed,True
tags,
versions,"191d81fc-0583-42f9-bcee-52cfad83fe47, eaa129cb-d505-46fd-95e2-91b09bc2c02c"
steps,ccfraudlstm


### Run Test Inference

Using the data from `sample_data_file`, a test inference will be made.

In [28]:
if arrowEnabled is True:
    sample_data_file = './smoke_test.df.json'
    response = pipeline.infer_from_file(sample_data_file)
else:
    sample_data_file = './smoke_test.json'
    response = pipeline.infer_from_file(sample_data_file)
display(response)

,model_name,model_version,pipeline_name,outputs,elapsed,time,original_data,check_failures,shadow_data
0,ccfraudlstm,cb4b62f1-c85a-4281-87d5-dba3cc41a834,cc-shadow,"[{'Float': {'v': 1, 'dim': [1, 1], 'data': [0.001497417688369751]}}]",120402,2023-03-01 19:30:17.491,None,0,"{'ccfraudrf': [{'Float': {'v': 1, 'dim': [1, 1], 'data': [1.0]}}], 'ccfraudxgb': [{'Float': {'v': 1, 'dim': [1, 1], 'data': [0.0005066990852355957]}}]}"


### View Pipeline Logs

With the inferences complete, we can retrieve the log data from the pipeline with the pipeline `logs` method.  Note that for **each** inference request, the logs return **one entry per model**.  For this example, for one inference request three log entries will be created.

In [29]:
pipeline.logs()

,time,message
0,2023-03-01 19:30:18.503,"{""model_name"":""ccfraudxgb"",""model_version"":""3575f5c1-77fe-4aeb-be80-dbe4c110dede"",""pipeline_name"":""cc-shadow"",""outputs"":[{""Float"":{""v"":1,""dim"":[1,1],""data"":[0.0005066990852355957]}}],""elapsed"":170504,""time"":1677699017491,""original_data"":null,""check_failures"":[],""shadow_data"":{}}"
1,2023-03-01 19:30:18.503,"{""model_name"":""ccfraudrf"",""model_version"":""f8060523-806d-4bc3-bece-b1afcd730949"",""pipeline_name"":""cc-shadow"",""outputs"":[{""Float"":{""v"":1,""dim"":[1,1],""data"":[1.0]}}],""elapsed"":640913,""time"":1677699017491,""original_data"":null,""check_failures"":[],""shadow_data"":{}}"
2,2023-03-01 19:30:18.503,"{""model_name"":""ccfraudlstm"",""model_version"":""cb4b62f1-c85a-4281-87d5-dba3cc41a834"",""pipeline_name"":""cc-shadow"",""outputs"":[{""Float"":{""v"":1,""dim"":[1,1],""data"":[0.001497417688369751]}}],""elapsed"":120402,""time"":1677699017491,""original_data"":null,""check_failures"":[],""shadow_data"":{""ccfraudrf"":[{""Float"":{""v"":1,""dim"":[1,1],""data"":[1.0]}}],""ccfraudxgb"":[{""Float"":{""v"":1,""dim"":[1,1],""data"":[0.0005066990852355957]}}]}}"


### View Logs Per Model

Another way of displaying the logs would be to specify the model.

For Arrow enabled Wallaroo instances, the log files are returned as a DataFrame, and the models can be specified by rows.  The following code will display the log data based based on the model name and the inference output for that specific model.

For arrow disabled Wallaroo instances, to view the inputs and results for the shadow deployed models, use the pipeline `logs_shadow_deploy()` method.  The results will be grouped by the inputs.

In [30]:
import json
if arrowEnabled is True:
    logs = pipeline.logs()
    for index, row in logs.iterrows():
        convertedjson = json.loads(row['message'])
        displayModelName = convertedjson['model_name']
        displayOutputs = str(convertedjson['outputs'][0]['Float']['data'][0])
        display([displayModelName,displayOutputs])
else:
    logs = pipeline.logs()
    for log in logs:
        display(log.model_name, log.output) 
    shadow_logs = pipeline.logs_shadow_deploy()
    display(shadow_logs)

['ccfraudxgb', '0.0005066990852355957']

['ccfraudrf', '1.0']

['ccfraudlstm', '0.001497417688369751']

### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the system.

In [32]:
pipeline.undeploy()

name,cc-shadow
created,2023-03-01 19:29:44.071174+00:00
last_updated,2023-03-01 19:29:49.728761+00:00
deployed,False
tags,
versions,"191d81fc-0583-42f9-bcee-52cfad83fe47, eaa129cb-d505-46fd-95e2-91b09bc2c02c"
steps,ccfraudlstm
